# Reproducing and trying to solve the bug in [this ticket](https://github.com/orgs/OpenMined/projects/81/views/1?pane=issue&itemId=28179093) - first notebook

In [1]:
import syft as sy
sy.requires(">=0.8,<0.8.1")

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /home/dk/Desktop/projects/PySyft


✅ The installed version of syft==0.8.1b2 matches the requirement >=0.8 and the requirement <0.8.1


In [2]:
node = sy.orchestra.launch(name="test-domain-1", port=8080, 
                           dev_mode=True, reset=True)

Starting test-domain-1 server on 0.0.0.0:8080

SQLite Store Path:
!open file:///tmp/7bca415d13ed1ec841f0d0aede098dbb.sqlite



INFO:     Started server process [106408]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8080 (Press CTRL+C to quit)


INFO:     127.0.0.1:42844 - "GET /api/v1/new/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:42844 - "POST /api/v1/new/login HTTP/1.1" 200 OK
INFO:     127.0.0.1:42844 - "GET /api/v1/new/api?verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f HTTP/1.1" 200 OK
INFO:     127.0.0.1:56726 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:56736 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:56738 - "GET /api/v1/new/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:56746 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:56738 - "GET /api/v1/new/api?verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f HTTP/1.1" 200 OK
INFO:     127.0.0.1:60620 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:60628 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:60632 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:60634 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
I

In [3]:
domain_client = node.login(email="info@openmined.org", 
                           password="changethis")

#### Construct a `custom policy`

In [4]:
from typing import List, Dict, Any, Optional

@sy.serializable()
class RepeatedCallPolicy(sy.CustomOutputPolicy):
    n_calls: int = 0
    downloadable_output_args: List[str] = []
    state: Dict[Any, Any] = {}

    def __init__(self, n_calls=1, downloadable_output_args: List[str] = None):
        self.downloadable_output_args = downloadable_output_args if downloadable_output_args is not None else []
        self.n_calls = n_calls + 1
        self.state = {"counts": 0}

    def public_state(self):
        return self.state["counts"]
        
    def apply_output(self, context, outputs):
        output_dict = {}
        if self.state["counts"] < self.n_calls:
            for output_arg in self.downloadable_output_args:
                output_dict[output_arg] = outputs[output_arg]

            self.state["counts"] += 1
        else:
            return None

        return output_dict

In [5]:
sy.CustomOutputPolicy??

In [5]:
policy = RepeatedCallPolicy(n_calls=1, downloadable_output_args=['y'])

In [6]:
print(f"{policy.n_calls = }")
print(f"{policy.downloadable_output_args = }")

policy.n_calls = 2
policy.downloadable_output_args = ['y']


In [7]:
policy.init_kwargs

{'n_calls': 1, 'downloadable_output_args': ['y']}

#### Make a pointer

In [8]:
import numpy as np
x = np.array([1,2,3])
x_pointer = sy.ActionObject.from_obj(x)
x_pointer

```python
Pointer
```
array([1, 2, 3])

In [9]:
domain_client.api.services.action.save(x_pointer)

<class 'syft.service.response.SyftSuccess'>: <class 'syft.service.action.numpy.NumpyArrayObject'> saved

In [10]:
x_pointer.id

<UID: 6a27a89e3fe043af84610328acbbe4a6>

#### Make a function with the `RepeatedCallPolicy`

In [11]:
@sy.syft_function(
    input_policy=sy.ExactMatch(x=x_pointer),
    output_policy=RepeatedCallPolicy(n_calls=10, 
                                     downloadable_output_args=['y']),
)
def func(x):
    return {"y": x+1}

In [45]:
sy.deserialize(sy.serialize(func, to_bytes=True), from_bytes=True)

```python
class UserCode:
  id: str = ece7407f26ab4583bd2483ad2b05077b
  node_uid: str = 7bca415d13ed1ec841f0d0aede098dbb
  user_verify_key: str = aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f
  raw_code: str = "@sy.syft_function(
    input_policy=sy.ExactMatch(x=x_pointer),
    output_policy=RepeatedCallPolicy(n_calls=10, 
                                     downloadable_output_args=['y']),
)
def func(x):
    return {"y": x+1}
"
  input_policy_type: str = <class 'syft.service.policy.policy.ExactMatch'>
  input_policy_init_kwargs: str = {NodeView(node_name='test-domain-1', verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f): {'x': <UID: 6a27a89e3fe043af84610328acbbe4a6>}}
  input_policy_state: str = b''
  output_policy_type: str = syft.service.policy.policy.UserPolicy
  output_policy_init_kwargs: str = {'n_calls': 10, 'downloadable_output_args': ['y']}
  output_policy_state: str = b''
  parsed_code: str = "def user_func_func_aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f_086ff1660a3fb90e99e47c9b3acf176ae21fe115cbf7fe524892403e48338f11(x):

    def func(x):
        return {'y': x + 1}
    result = func(x=x)
    return result"
  service_func_name: str = "func"
  unique_func_name: str = "user_func_func_aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f_086ff1660a3fb90e99e47c9b3acf176ae21fe115cbf7fe524892403e48338f11"
  user_unique_func_name: str = "user_func_func_aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f"
  code_hash: str = "086ff1660a3fb90e99e47c9b3acf176ae21fe115cbf7fe524892403e48338f11"
  signature: str = (x)
  status: str = {NodeView(node_name='test-domain-1', verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f): <UserCodeStatus.SUBMITTED: 'submitted'>}
  input_kwargs: str = ['x']
  enclave_metadata: str = None

```

#### Request code execution

In [12]:
domain_client.api.services.code.request_code_execution(func)

```python
class Request:
  id: str = 875a51e6a5294b318584600ac06741bd
  requesting_user_verify_key: str = aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f
  approving_user_verify_key: str = None
  request_time: str = 2023-05-17 23:13:13
  approval_time: str = None
  status: str = RequestStatus.PENDING
  node_uid: str = 7bca415d13ed1ec841f0d0aede098dbb
  request_hash: str = "112d9ee67171cb80aeaa2ba95972f424428e110623aea89c81d9fb314c570c0f"
  changes: str = [syft.service.request.request.UserCodeStatusChange]

```

In [13]:
from syft import MessageStatus
messages = domain_client.api.services.messages.get_all_for_status(MessageStatus.UNDELIVERED)
messages

,type,id,subject,status,created_at,linked_obj
0,syft.service.message.messages.Message,502f12d170604417a770cf8af718a16b,Approval Request,MessageStatus.UNDELIVERED,2023-05-17 23:13:13,<<class 'syft.service.request.request.Request'...


In [14]:
from syft.service.request.request import ObjectMutation, UserCodeStatusChange, Request
func = None
request = None
for message in messages:
    req = message.linked_obj.resolve
    if isinstance(req, Request):
        for change in req.changes:
            if isinstance(change, UserCodeStatusChange):
                user_code = change.linked_obj.resolve
                if "func" in user_code.service_func_name:
                    func = user_code
                    request = req

In [15]:
request.changes

,type,id
0,syft.service.request.request.UserCodeStatusChange,95242c3be6114cee86835506cffc78e1


In [16]:
print(f"{x_pointer = }")
result = func.unsafe_function(x=x_pointer)
result

x_pointer = Pointer:
[1 2 3]


{'y': array([2, 3, 4])}

In [17]:
final_result = request.accept_by_depositing_result(result) 
final_result

<class 'syft.service.response.SyftSuccess'>: Request 875a51e6a5294b318584600ac06741bd changes applied

In [18]:
res = domain_client.api.services.code.func(x=x_pointer)
res

Pointer:
{'y': array([2, 3, 4])}

In [19]:
domain_client.api.services.request

,type,id,request_time,status,changes
0,syft.service.request.request.Request,875a51e6a5294b318584600ac06741bd,2023-05-17 23:13:13,RequestStatus.PENDING,[syft.service.request.request.UserCodeStatusCh...
1,syft.service.request.request.Request,a41d378761514624a3d5073e04324fb7,2023-05-17 23:13:19,RequestStatus.PENDING,"[syft.service.request.request.ObjectMutation, ..."


In [20]:
domain_client.api.services.messages

,type,id,subject,status,created_at,linked_obj
0,syft.service.message.messages.Message,502f12d170604417a770cf8af718a16b,Approval Request,MessageStatus.UNDELIVERED,2023-05-17 23:13:13,<<class 'syft.service.request.request.Request'...
1,syft.service.message.messages.Message,76f74b53bd02493c99f528e342c69184,Approval Request,MessageStatus.UNDELIVERED,2023-05-17 23:13:19,<<class 'syft.service.request.request.Request'...


In [21]:
type(policy)

__main__.RepeatedCallPolicy

In [40]:
policy.__class__.__module__

'__main__'

In [43]:
RepeatedCallPolicy.__class__.__module__

'syft.service.policy.policy'

get_fully_qualified_name() in `util.py` should return `syft.service.policy.policy.RepeatedCallPolicy`

#### Let's try serializing a `CustomOutputPolicy`

In [23]:
import syft as sy
from syft.service.policy.policy import CustomPolicy
from syft.service.policy.policy import CustomOutputPolicy

We can `serialize` and `deserialize` the `CustomPolicy`

In [39]:
byte = sy.serialize(CustomPolicy, to_bytes=True)
back = sy.deserialize(byte, from_bytes=True)
back == CustomPolicy

True

In [37]:
byte = sy.serialize(CustomOutputPolicy, to_bytes=True)
back = sy.deserialize(byte, from_bytes=True)
back

syft.service.policy.policy.CustomOutputPolicy

In [26]:
CustomPolicy

syft.service.policy.policy.CustomPolicy

In [24]:
sy.deserialize(sy.serialize(policy, to_bytes=True), 
               from_bytes=True)

In [36]:
RepeatedCallPolicy.__class__.__module__

syft.service.policy.policy.CustomPolicy

In [24]:
import syft as sy

class a(type):
    print('a')
    
class b(metaclass=a):
    print('b')
    
# a == sy.deserialize(sy.serialize(a, to_bytes=True), 
#                from_bytes=True)
b == sy.deserialize(sy.serialize(b, to_bytes=True), 
               from_bytes=True)

a
b


Exception: __main__.a not in TYPE_BANK